# California House Price Prediction

This notebook trains a Random Forest model to predict median house prices in California.

**Dataset**: California Housing (1990 Census)
- 20,640 samples from California districts
- Target: Median house value (in $100,000s)

**Features**:
- `MedInc`: Median household income ($10,000s)
- `HouseAge`: Median age of houses (years)
- `AveRooms`: Average rooms per household
- `AveBedrms`: Average bedrooms per household  
- `Population`: Block group population
- `AveOccup`: Average household size
- `Latitude`: Geographic latitude
- `Longitude`: Geographic longitude


In [ ]:
!pip install pandas numpy scikit-learn mlflow

In [ ]:
import os
import sys
import argparse
import json
import tempfile
import numpy as np
import pandas as pd
from datetime import datetime

# MLflow imports
import mlflow
import mlflow.sklearn
from mlflow import set_tracking_uri, set_experiment
from mlflow.client import MlflowClient
from mlflow.models import infer_signature

# Scikit-learn imports
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [ ]:
def setup_mlflow(mlflow_uri: str, username: str, password: str) -> MlflowClient:
    """Configure MLflow tracking and return client."""
    os.environ["MLFLOW_TRACKING_USERNAME"] = username
    os.environ["MLFLOW_TRACKING_PASSWORD"] = password
    
    set_tracking_uri(mlflow_uri)
    client = MlflowClient(mlflow_uri)
    
    print(f"✅ MLflow tracking URI: {mlflow_uri}")
    return client


def load_and_prepare_data():
    """Load California housing dataset and prepare train/test splits."""
    print("\n" + "=" * 80)
    print("📊 LOADING DATASET")
    print("=" * 80)
    
    data = fetch_california_housing(as_frame=True)
    X = data.data
    y = data.target
    
    print(f"Dataset: California Housing")
    print(f"Samples: {X.shape[0]:,}")
    print(f"Features: {X.shape[1]}")
    print(f"\nFeature names:")
    for i, col in enumerate(X.columns, 1):
        print(f"  {i}. {col}")
    
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, shuffle=True
    )
    
    print(f"\nTrain samples: {X_train.shape[0]:,}")
    print(f"Test samples: {X_test.shape[0]:,}")
    print(f"Target range: ${y.min():.2f} - ${y.max():.2f} (hundred thousands)")
    
    return X_train, X_test, y_train, y_test, X.columns.tolist()


def train_model(X_train, y_train, X_test, y_test, hyperparams: dict):
    """Train Random Forest model and return predictions."""
    print("\n" + "=" * 80)
    print("🚀 TRAINING MODEL")
    print("=" * 80)
    
    print("Hyperparameters:")
    for key, value in hyperparams.items():
        print(f"  {key}: {value}")
    
    model = RandomForestRegressor(**hyperparams)
    model.fit(X_train, y_train)
    
    print("✅ Training completed!")
    
    # Predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    return model, y_train_pred, y_test_pred


def calculate_metrics(y_true, y_pred, dataset_name="Test"):
    """Calculate and return evaluation metrics."""
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    return {
        f"{dataset_name.lower()}_rmse": rmse,
        f"{dataset_name.lower()}_mae": mae,
        f"{dataset_name.lower()}_r2": r2
    }


def log_to_mlflow(model, X_train, y_train, X_test, y_test, 
                  y_train_pred, y_test_pred, hyperparams, feature_names):
    """Log model, parameters, and metrics to MLflow."""
    print("\n" + "=" * 80)
    print("📝 LOGGING TO MLFLOW")
    print("=" * 80)
    
    # Log hyperparameters
    for key, value in hyperparams.items():
        mlflow.log_param(key, value)
    
    # Calculate and log metrics
    train_metrics = calculate_metrics(y_train, y_train_pred, "Train")
    test_metrics = calculate_metrics(y_test, y_test_pred, "Test")
    all_metrics = {**train_metrics, **test_metrics}
    
    for metric_name, metric_value in all_metrics.items():
        mlflow.log_metric(metric_name, metric_value)
    
    print("\n📈 Model Performance:")
    print(f"  Training RMSE: {train_metrics['train_rmse']:.4f}")
    print(f"  Training R²: {train_metrics['train_r2']:.4f}")
    print(f"  Test RMSE: {test_metrics['test_rmse']:.4f}")
    print(f"  Test MAE: {test_metrics['test_mae']:.4f}")
    print(f"  Test R²: {test_metrics['test_r2']:.4f}")

    # Create output with explicit column name
    y_pred_df = pd.DataFrame(model.predict(X_train), columns=["predicted_price"])

    # Create model signature
    signature = infer_signature(X_train, y_pred_df)
    input_example = X_test.head(1)
    
    # Save and log model
    with tempfile.TemporaryDirectory() as tmpdir:
        local_model_path = os.path.join(tmpdir, "model")
        
        mlflow.sklearn.save_model(
            model,
            local_model_path,
            signature=signature,
            input_example=input_example
        )
        
        mlflow.log_artifacts(local_model_path, artifact_path="model")
        print("✅ Model artifacts logged successfully!")
    
    return all_metrics


def create_sample_payload(X_test, y_test, model, feature_names):
    """Create realistic sample prediction payload."""
    # Get a sample with median income around 3-4 (middle class)
    mid_income_samples = X_test[(X_test['MedInc'] >= 3) & (X_test['MedInc'] <= 5)]
    if len(mid_income_samples) > 0:
        sample_idx = mid_income_samples.index[0]
        sample = X_test.loc[sample_idx]
        actual_price = y_test.loc[sample_idx]
    else:
        sample = X_test.iloc[0]
        actual_price = y_test.iloc[0]
    
    predicted_price = model.predict(sample.values.reshape(1, -1))[0]
    
    return {
        "features": sample.to_dict(),
        "actual_price": float(actual_price),
        "predicted_price": float(predicted_price)
    }


def register_model(client: MlflowClient, model_name: str, run_id: str, experiment_id: str):
    """Register model in MLflow Model Registry."""
    print("\n" + "=" * 80)
    print("📦 REGISTERING MODEL")
    print("=" * 80)
    
    model_uri = f"runs:/{run_id}/model"
    
    # Create registered model if it doesn't exist
    try:
        client.get_registered_model(model_name)
        print(f"ℹ️  Model '{model_name}' already exists in registry")
    except Exception:
        try:
            client.create_registered_model(model_name)
            print(f"✅ Created registered model: {model_name}")
        except Exception as e:
            print(f"⚠️  Could not create registered model: {e}")
    
    # Create model version
    try:
        result = client.create_model_version(
            name=model_name,
            source=model_uri,
            run_id=run_id
        )
        print(f"✅ Model version registered successfully!")
        print(f"   Model Name: {model_name}")
        print(f"   Version: {result.version}")
        print(f"   Run ID: {run_id}")
        return result.version
    except Exception as e:
        print(f"⚠️  Model registration failed (model still usable via run URI): {e}")
        print(f"   You can deploy using: mlflow-artifacts:/{experiment_id}/{run_id}/artifacts/model")
        return None


def print_deployment_info(run_id: str, experiment_id: str, sample_payload: dict):
    """Print deployment instructions and sample payloads."""
    print("\n" + "=" * 80)
    print("🎉 TRAINING COMPLETE!")
    print("=" * 80)
    
    print(f"\n📋 Run Information:")
    print(f"  Run ID: {run_id}")
    print(f"  Experiment ID: {experiment_id}")
    print(f"  Model URI: mlflow-artifacts:/{experiment_id}/{run_id}/artifacts/model")
    
    print("\n" + "=" * 80)
    print("🚀 DEPLOYMENT PAYLOAD (deploy-model API)")
    print("=" * 80)
    
    deploy_payload = {
        "serve_name": "house-pricing-model",
        "model_uri": f"mlflow-artifacts:/{experiment_id}/{run_id}/artifacts/model",
        "env": "local",
        "cores": 2,
        "memory": 4,
        "node_capacity": "spot",
        "min_replicas": 1,
        "max_replicas": 3
    }
    
    print(json.dumps(deploy_payload, indent=2))
    
    print("\n" + "=" * 80)
    print("📊 SAMPLE PREDICTION PAYLOAD")
    print("=" * 80)
    
    predict_payload = {
        "features": sample_payload["features"]
    }
    
    print(json.dumps(predict_payload, indent=2))
    
    print(f"\n💡 Expected Output:")
    print(f"  Actual Price: ${sample_payload['actual_price']:.2f} (hundred thousands) = ${sample_payload['actual_price'] * 100000:.0f}")
    print(f"  Model Prediction: ${sample_payload['predicted_price']:.2f} (hundred thousands) = ${sample_payload['predicted_price'] * 100000:.0f}")
    
    print("\n" + "=" * 80)
    print("📝 FEATURE DESCRIPTIONS")
    print("=" * 80)
    features = sample_payload["features"]
    print(f"""
  MedInc:      ${features['MedInc']:.2f} × $10,000 = ${features['MedInc'] * 10000:.0f} median income
  HouseAge:    {features['HouseAge']:.0f} years old
  AveRooms:    {features['AveRooms']:.1f} rooms per household
  AveBedrms:   {features['AveBedrms']:.1f} bedrooms per household
  Population:  {features['Population']:.0f} people in block group
  AveOccup:    {features['AveOccup']:.1f} people per household
  Latitude:    {features['Latitude']:.2f}°N
  Longitude:   {features['Longitude']:.2f}°W
    """)


def main():
    parser = argparse.ArgumentParser(description="Train California House Pricing Model")
    parser.add_argument(
        "--mlflow-uri",
        default="http://darwin-mlflow-lib.darwin.svc.cluster.local:8080",
        help="MLflow tracking URI"
    )
    parser.add_argument(
        "--username",
        default="dhruv.mehta@dream11.com",
        help="MLflow username"
    )
    parser.add_argument(
        "--password",
        default="dhruv.mehta@dream11.com",
        help="MLflow password"
    )
    parser.add_argument(
        "--experiment-name",
        default="house_pricing",
        help="MLflow experiment name"
    )
    parser.add_argument(
        "--model-name",
        default="HousePricingRF",
        help="Registered model name"
    )
    
    args, _ = parser.parse_known_args()
    
    print("\n" + "=" * 80)
    print("🏠 CALIFORNIA HOUSE PRICING MODEL TRAINING")
    print("=" * 80)
    print(f"Started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    # Setup MLflow
    client = setup_mlflow(args.mlflow_uri, args.username, args.password)
    set_experiment(experiment_name=args.experiment_name)
    print(f"✅ Experiment: {args.experiment_name}")
    
    # Load data
    X_train, X_test, y_train, y_test, feature_names = load_and_prepare_data()
    
    # Define hyperparameters
    hyperparams = {
        "n_estimators": 100,
        "max_depth": 15,
        "min_samples_split": 5,
        "min_samples_leaf": 2,
        "random_state": 42,
        "n_jobs": -1
    }
    
    # Start MLflow run
    with mlflow.start_run(run_name=f"rf_house_pricing_{datetime.now().strftime('%Y%m%d_%H%M%S')}"):
        # Train model
        model, y_train_pred, y_test_pred = train_model(
            X_train, y_train, X_test, y_test, hyperparams
        )
        
        # Log to MLflow
        metrics = log_to_mlflow(
            model, X_train, y_train, X_test, y_test,
            y_train_pred, y_test_pred, hyperparams, feature_names
        )
        
        # Get run information
        run_id = mlflow.active_run().info.run_id
        experiment_id = mlflow.active_run().info.experiment_id
        
        # Create sample payload
        sample_payload = create_sample_payload(X_test, y_test, model, feature_names)
    
    # Register model (outside of run context)
    version = register_model(client, args.model_name, run_id, experiment_id)
    
    # Print deployment information
    print_deployment_info(run_id, experiment_id, sample_payload)
    
    print("\n✅ Script completed successfully!")
    print("=" * 80 + "\n")


if __name__ == "__main__":
    main()